In [ ]:
import socket, cv2, pickle,struct,imutils
import threading
server_socket = socket.socket()
server_socket.setsockopt(socket.SOL_SOCKET , socket.SO_REUSEADDR, 1)
host_ip = '192.168.43.118'
print('HOST IP:',host_ip)
port = 1234
socket_address = (host_ip,port)
# Socket Bind
server_socket.bind(socket_address)
# Socket Listen
server_socket.listen()
print("LISTENING AT:",socket_address)
def sending_server(client_socket,addr):
    print('GOT CONNECTION FROM:',addr)
    if client_socket:
        vid = cv2.VideoCapture(0)
        
        while(vid.isOpened()):
            try:
                img,frame = vid.read()
                frame = imutils.resize(frame,width=520)
                a = pickle.dumps(frame)
                message = struct.pack("Q",len(a))+a
                client_socket.sendall(message)
            except ConnectionAbortedError:
                vid.release()
                client_socket.close()
                break
            
def receving_from_client():
    data = b""
    payload_size = struct.calcsize("Q")
    while True:
        while len(data) < payload_size:
            packet = client_socket.recv(4*1024) # 4K
            if not packet: break
            data+=packet
        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("Q",packed_msg_size)[0]
        
        while len(data) < msg_size:
            data += client_socket.recv(4*1024)
        frame_data = data[:msg_size]
        data  = data[msg_size:]
        frame = pickle.loads(frame_data) 
        cv2.imshow('Client send to server VIDEO',frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 13:
            client_socket.close()
            break
    cv2.destroyAllWindows()
while True:
    client_socket, addr = server_socket.accept()
    if client_socket:
        t2 = threading.Thread(target=receving_from_client)
        t2.start()
        t1 = threading.Thread(target=sending_server,args=(client_socket,addr))
        t1.start()